In [1]:
import os
import torch
import librosa

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datasetsmelmean import AudioDataset

import sklearn.metrics as met
# hyper parameters
MAX_EPOCH = 100

/home/wew016/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/wew016/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
# path to urban sound 8k
data_root = "/home/wew016/UrbanSound8K/"
# path to label
label_path = "/home/wew016/UrbanSound8K/metadata/UrbanSound8K.csv"        

In [3]:
# initialize dataset (feature can be "mfcc" or "spec")
audio_dataset = AudioDataset(10, DataRoot=data_root, LabelPath=label_path, feature="mel_mean", mode="train")

verify mel_mean feature success


In [4]:
# initialize dataloader
data_loader = torch.utils.data.DataLoader(audio_dataset, batch_size=len(audio_dataset), shuffle=True, num_workers=1)

for idx, data in enumerate(data_loader):
        x_train, y_train = data
        print(x_train.shape)
        print(y_train.shape) 

torch.Size([7895, 128])
torch.Size([7895])


In [5]:
x_train=x_train.numpy()
x_train.shape

(7895, 128)

In [6]:
y_train=y_train.numpy()
y_train.shape

(7895,)

In [7]:
audio_dataset_test = AudioDataset(10, DataRoot=data_root, LabelPath=label_path, feature="mel_mean", mode="test")

verify mel_mean feature success


In [8]:
# initialize dataloader
data_loader_test = torch.utils.data.DataLoader(audio_dataset_test, batch_size=len(audio_dataset_test), shuffle=True, num_workers=1)

for idx, data in enumerate(data_loader_test):
        x_test, y_test = data
        print(x_test.shape)
        print(y_test.shape)    

torch.Size([837, 128])
torch.Size([837])


In [9]:
x_test=x_test.numpy()
x_test.shape

(837, 128)

In [10]:
y_test=y_test.numpy()
y_test.shape

(837,)

<font color=red size=5>
XGBOOST Model
</font>

In [11]:
from xgboost import XGBClassifier

In [12]:
model_xgboost = XGBClassifier(learning_rate =0.2, 
                      n_estimators=150, 
                      max_depth=4,
                      num_class=10,    
                      min_child_weight=5, 
                      gamma=0.1, 
                      subsample=0.5, 
                      colsample_bytree=0.5,
                      objective='multi:softmax', 
                      seed=50)

In [13]:
model_xgboost.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=150, n_jobs=0, num_class=10, num_parallel_tree=1,
              objective='multi:softprob', random_state=50, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=50, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
pred_train_xgboost = model_xgboost.predict(x_train)
met.accuracy_score(y_train,pred_train_xgboost)

0.9956934768841038

In [15]:
pred_test_xgboost = model_xgboost.predict(x_test)

In [16]:
y_test

array([1, 7, 9, 4, 5, 2, 4, 2, 4, 5, 3, 5, 4, 5, 2, 4, 9, 2, 2, 2, 3, 8,
       7, 0, 4, 1, 3, 3, 2, 2, 7, 2, 8, 5, 3, 9, 7, 3, 4, 9, 2, 3, 9, 0,
       8, 4, 0, 1, 2, 5, 5, 5, 0, 1, 3, 4, 6, 6, 5, 4, 3, 0, 2, 8, 9, 0,
       3, 5, 9, 5, 9, 8, 0, 2, 9, 8, 9, 0, 5, 8, 4, 4, 2, 4, 0, 3, 2, 9,
       0, 5, 5, 8, 2, 0, 0, 3, 6, 4, 4, 5, 2, 5, 1, 4, 5, 2, 0, 8, 7, 3,
       3, 7, 0, 5, 4, 0, 0, 7, 9, 7, 7, 0, 8, 3, 8, 9, 8, 1, 9, 7, 7, 3,
       2, 0, 7, 9, 7, 0, 9, 5, 4, 7, 4, 2, 5, 4, 8, 0, 4, 2, 0, 5, 4, 7,
       0, 9, 1, 2, 8, 5, 3, 9, 0, 5, 8, 4, 3, 8, 8, 4, 9, 7, 2, 4, 7, 3,
       3, 9, 6, 2, 0, 0, 9, 9, 8, 1, 5, 0, 5, 7, 8, 0, 1, 5, 3, 8, 3, 4,
       7, 2, 4, 3, 4, 4, 2, 4, 9, 9, 2, 8, 2, 8, 2, 4, 8, 5, 5, 9, 5, 7,
       8, 2, 8, 2, 6, 6, 4, 3, 3, 9, 2, 9, 4, 2, 2, 9, 0, 8, 6, 7, 8, 8,
       3, 5, 9, 3, 2, 4, 0, 3, 9, 7, 7, 9, 4, 7, 9, 0, 6, 3, 0, 4, 3, 8,
       9, 8, 7, 3, 5, 0, 9, 8, 8, 5, 4, 0, 9, 4, 8, 4, 9, 9, 9, 4, 0, 4,
       8, 8, 1, 4, 5, 3, 0, 2, 6, 5, 2, 7, 6, 3, 9,

In [17]:
pred_test_xgboost

array([1, 9, 9, 4, 2, 2, 9, 3, 9, 5, 0, 9, 9, 9, 2, 3, 9, 2, 2, 8, 3, 8,
       4, 9, 4, 1, 9, 3, 2, 2, 4, 2, 8, 2, 2, 9, 4, 3, 9, 9, 2, 3, 9, 2,
       8, 4, 0, 6, 3, 2, 2, 2, 0, 1, 3, 4, 2, 2, 5, 4, 9, 2, 2, 8, 9, 9,
       3, 5, 6, 2, 5, 2, 9, 2, 9, 2, 9, 0, 2, 8, 4, 9, 2, 9, 0, 3, 2, 9,
       0, 2, 2, 0, 2, 2, 0, 3, 6, 4, 9, 5, 2, 2, 9, 4, 2, 2, 2, 2, 4, 3,
       3, 4, 2, 2, 7, 2, 9, 7, 2, 4, 4, 9, 0, 3, 2, 9, 3, 1, 2, 4, 4, 3,
       8, 9, 4, 9, 4, 9, 9, 9, 7, 4, 3, 2, 2, 4, 2, 9, 4, 2, 2, 2, 4, 4,
       9, 9, 1, 2, 8, 0, 8, 0, 9, 2, 2, 4, 3, 2, 8, 7, 7, 7, 8, 4, 4, 4,
       3, 7, 6, 2, 0, 5, 9, 2, 8, 1, 2, 9, 2, 2, 8, 0, 7, 9, 3, 0, 9, 9,
       4, 2, 4, 2, 4, 4, 8, 9, 2, 9, 2, 0, 2, 0, 8, 2, 3, 2, 2, 9, 2, 4,
       8, 8, 2, 1, 2, 6, 9, 3, 5, 9, 2, 2, 4, 2, 2, 9, 0, 8, 6, 7, 0, 2,
       7, 9, 2, 3, 8, 6, 9, 3, 9, 7, 4, 3, 7, 6, 2, 9, 4, 3, 0, 7, 3, 2,
       9, 2, 4, 2, 5, 0, 9, 2, 8, 2, 8, 0, 9, 9, 8, 6, 9, 9, 9, 2, 9, 4,
       3, 8, 1, 7, 2, 3, 0, 2, 6, 5, 2, 7, 4, 3, 5,

In [18]:
met.accuracy_score(y_test,pred_test_xgboost)

0.45639187574671447

In [208]:
met.confusion_matrix(y_test,pred_test_xgboost)

array([[ 6,  0, 36,  8,  0, 29, 11, 10,  0,  0],
       [ 0, 40,  0,  1,  1,  0,  1,  0,  0,  0],
       [ 5,  0, 58, 19,  2,  1,  2,  1,  0, 12],
       [ 0,  0, 12, 66,  1,  0,  1,  0, 11,  9],
       [ 0,  0,  8, 27, 38,  1,  5,  3, 11,  7],
       [ 3,  0, 14,  8, 22, 18, 18, 10,  0, 14],
       [ 0,  0,  1,  0,  2,  0, 26,  4,  0,  3],
       [ 6,  0,  1,  0, 14,  3,  0, 29,  0, 67],
       [ 4,  0,  6, 10,  0,  3,  0,  7, 72, 17],
       [ 7,  0, 36,  9,  1,  5,  0,  5,  1, 36]])

<font color=red size=5>
SVM Model
</font>

In [19]:
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
svr = SVC()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 2, 4], 'gamma':[0.125, 0.25, 0.5 ,1, 2, 4]}
clf = GridSearchCV(svr, parameters, scoring='f1_samples')
clf.fit(x_train, y_train)
print('The parameters of the best model are: ')
print(clf.best_params_)

In [21]:
model_svm = sklearn.svm.SVC(C=1000, 
                            kernel='rbf', 
                            degree=3, 
                            gamma='auto', 
                            coef0=0.0, 
                            shrinking=True, 
                            probability=False, 
                            tol=0.001, 
                            cache_size=200, 
                            class_weight=None, 
                            verbose=False, 
                            max_iter=-1, 
                            decision_function_shape=None, 
                            random_state=None)

In [22]:
model_svm.fit(x_train,y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
pred_train_svm = model_svm.predict(x_train)
met.accuracy_score(y_train,pred_train_svm)

0.9422419252691577

In [24]:
pred_test_svm = model_svm.predict(x_test)
met.accuracy_score(y_test,pred_test_svm)

0.31899641577060933

<font color=red size=5>
RandomForest Model
</font>

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
model_RF = RandomForestClassifier(n_estimators=100 ,criterion = "entropy", bootstrap= False)

In [27]:
model_RF.fit(x_train,y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [28]:
pred_train_RF = model_RF.predict(x_train)
met.accuracy_score(y_train,pred_train_RF)

1.0

In [29]:
pred_test_RF = model_RF.predict(x_test)
met.accuracy_score(y_test,pred_test_RF)

0.45161290322580644

<font color=red size=5>
KNN Model
</font>

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
model_KNN = KNeighborsClassifier(n_neighbors=10, 
                                 weights='uniform',
                                 algorithm='auto',
                                 leaf_size=30, 
                                 p=2, 
                                 metric='minkowski', 
                                 metric_params=None, 
                                 n_jobs=None)

In [32]:
model_KNN.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [33]:
pred_train_KNN = model_KNN.predict(x_train)
met.accuracy_score(y_train,pred_train_KNN)

0.7336288790373654

In [34]:
pred_test_KNN = model_KNN.predict(x_test)
met.accuracy_score(y_test,pred_test_KNN)

0.3847072879330944

<font color=red size=5>
KNN Model
</font>